In [ ]:
# import required libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
nltk.download('punkt')
import tensorflow_hub as hub
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # convert to lowercase
    text = text.lower()
    # remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # remove stopwords and lemmatize
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    text = ' '.join(tokens)
    return text


# load the dataset
data = pd.read_csv('dataset.csv')

# preprocess the data
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X = np.array([embed([preprocess_text(q + ' ' + a)]).numpy().flatten() for q, a in zip(data['question'], data['student_answer'])])
y = data['score_avg']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# create the model and fit it to the training data
model = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu', solver='adam', max_iter=500, random_state=42)
model.fit(X_train, y_train)

# evaluate the model on the testing set
y_pred = model.predict(X_test)

# compute evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# print evaluation metrics
print('R-squared:', r2)
print('Mean absolute error:', mae)
print('Mean squared error:', mse)
print('Root mean squared error:', rmse)

# deploy the model to production
def predict_relevance(question, desired_answer, student_answer):
    # preprocess the text
    question = preprocess_text(question)
    desired_answer = preprocess_text(desired_answer)
    student_answer = preprocess_text(student_answer)

    # concatenate question and student_answer texts
    input_text = question + ' ' + student_answer
    
    # encode input text using Universal Sentence Encoder
    input_emb = embed([input_text]).numpy().flatten()
    
    # predict the relevance score of the input text
    score = model.predict([input_emb])[0]
    print(score)
    # compute cosine similarity between desired_answer and student_answer texts
    similarity = cosine_similarity(embed([desired_answer]).numpy(), embed([student_answer]).numpy())[0][0]
    print("Cosine ", similarity)
    
    # adjust score based on similarity
    if similarity < 0.4:
        score -= 1
    elif similarity >= 0.4 and similarity < 0.8:
        score += 1
    else:
        score += 2
    
    # ensure score is between 0 and 5
    score = min(max(score, 0), 5)
    
    return score


question = "What is the difference between an array declared as static, and one that is not?"
desired_answer = "The arrays declared as static live throughout the life of the program; that is, they are initialized only once, when the function that declares the array it is first called."
student_answer = "a static array will store the new values that were assigned to each of its elements. meaning if you call a function twice it will use the last values that were returned the first time.  if you don't declare it static then the new values will not be stored and will be reset to their original value"
new_answer = "adt adt adt adt adt adt"
newAnswer = "An array declared as static has a fixed size and retains its value between function calls, while a non-static array does not retain its value and its size can change during runtime."
print("=================== Irrelevant answers ==========================")
print(predict_relevance(question, desired_answer, new_answer))
print()
print(predict_relevance(question, desired_answer, "I don't know"))
print()
print(predict_relevance(question, desired_answer, "static one stays while non static one leaves"))
print()
print(predict_relevance(question, desired_answer, "one is memory allocated while other is storage allocated"))
print()
print(predict_relevance(question, desired_answer, "The static array is like a classic car that never goes out of style, while the non-static array is like a trendy new smartphone that quickly becomes outdated."))
print()
print(predict_relevance(question, desired_answer, "static array, non-static array, static array non-static array static array non-static array static array non-static array static array non-static array static array non-static array "))
print()
print(predict_relevance(question, desired_answer, "An array is something which starts with square brackets and all values inside it are seaperated by commas thats why it is static"))
print()
print(predict_relevance(question, desired_answer, "The static array is something that is not non static. Thats why it is called as static array."))
print()


print("=================== relevant answers ==========================")
print(predict_relevance(question, desired_answer, student_answer))
print()
print(predict_relevance(question, desired_answer, newAnswer))
print()
print(predict_relevance(question, desired_answer, "A static array persists throughout the lifetime of a program, while a non-static array is deallocated once it goes out of scope."))
print()
print(predict_relevance(question, desired_answer, "A static array is initialized to zero by default, while a non-static array has an undefined initial value unless explicitly initialized."))
print()
print(predict_relevance(question, desired_answer, "A static array can be accessed from any function within a file, while a non-static array is only accessible within the function in which it is declared."))
print()
print(predict_relevance(question, desired_answer, "A static array can be initialized only once, at compile time, while a non-static array can be initialized multiple times, including at runtime."))
print()
print(predict_relevance(question, desired_answer, "A static array can have a fixed size that is determined at compile time, while a non-static array can have a variable size that is determined at runtime."))
print()


In [ ]:
# import required libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
nltk.download('punkt')
import tensorflow_hub as hub
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # convert to lowercase
    text = text.lower()
    # remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # remove stopwords and lemmatize
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    text = ' '.join(tokens)
    return text


# load the dataset
data = pd.read_csv('dataset.csv')

# preprocess the data
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X = np.array([embed([preprocess_text(q + ' ' + a)]).numpy().flatten() for q, a in zip(data['question'], data['student_answer'])])
y = data['score_avg']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# create the model and fit it to the training data
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
model.fit(X_train, y_train)

# evaluate the model on the testing set
y_pred = model.predict(X_test)

# compute evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# print evaluation metrics
print('R-squared:', r2)
print('Mean absolute error:', mae)
print('Mean squared error:', mse)
print('Root mean squared error:', rmse)

# deploy the model to production
def predict_relevance(question, desired_answer, student_answer):
    # preprocess the text
    question = preprocess_text(question)
    desired_answer = preprocess_text(desired_answer)
    student_answer = preprocess_text(student_answer)

    # concatenate question and student_answer texts
    input_text = question + ' ' + student_answer
    
    # encode input text using Universal Sentence Encoder
    input_emb = embed([input_text]).numpy().flatten()
    
    # predict the relevance score of the input text
    score = model.predict([input_emb])[0]
    print(score)
    # compute cosine similarity between desired_answer and student_answer texts
    similarity = cosine_similarity(embed([desired_answer]).numpy(), embed([student_answer]).numpy())[0][0]
    print("Cosine ", similarity)
    
    # adjust score based on similarity
    if similarity < 0.4:
        score -= 1
    elif similarity >= 0.4 and similarity < 0.8:
        score += 1
    else:
        score += 2
    
    # ensure score is between 0 and 5
    score = min(max(score, 0), 5)
    
    return score


question = "What is the difference between an array declared as static, and one that is not?"
desired_answer = "The arrays declared as static live throughout the life of the program; that is, they are initialized only once, when the function that declares the array it is first called."
student_answer = "a static array will store the new values that were assigned to each of its elements. meaning if you call a function twice it will use the last values that were returned the first time.  if you don't declare it static then the new values will not be stored and will be reset to their original value"
new_answer = "adt adt adt adt adt adt"
newAnswer = "An array declared as static has a fixed size and retains its value between function calls, while a non-static array does not retain its value and its size can change during runtime."
print("=================== Irrelevant answers ==========================")
print(predict_relevance(question, desired_answer, new_answer))
print()
print(predict_relevance(question, desired_answer, "I don't know"))
print()
print(predict_relevance(question, desired_answer, "static one stays while non static one leaves"))
print()
print(predict_relevance(question, desired_answer, "one is memory allocated while other is storage allocated"))
print()
print(predict_relevance(question, desired_answer, "The static array is like a classic car that never goes out of style, while the non-static array is like a trendy new smartphone that quickly becomes outdated."))
print()
print(predict_relevance(question, desired_answer, "static array, non-static array, static array non-static array static array non-static array static array non-static array static array non-static array static array non-static array "))
print()
print(predict_relevance(question, desired_answer, "An array is something which starts with square brackets and all values inside it are seaperated by commas thats why it is static"))
print()
print(predict_relevance(question, desired_answer, "The static array is something that is not non static. Thats why it is called as static array."))
print()


print("=================== relevant answers ==========================")
print(predict_relevance(question, desired_answer, student_answer))
print()
print(predict_relevance(question, desired_answer, newAnswer))
print()
print(predict_relevance(question, desired_answer, "A static array persists throughout the lifetime of a program, while a non-static array is deallocated once it goes out of scope."))
print()
print(predict_relevance(question, desired_answer, "A static array is initialized to zero by default, while a non-static array has an undefined initial value unless explicitly initialized."))
print()
print(predict_relevance(question, desired_answer, "A static array can be accessed from any function within a file, while a non-static array is only accessible within the function in which it is declared."))
print()
print(predict_relevance(question, desired_answer, "A static array can be initialized only once, at compile time, while a non-static array can be initialized multiple times, including at runtime."))
print()
print(predict_relevance(question, desired_answer, "A static array can have a fixed size that is determined at compile time, while a non-static array can have a variable size that is determined at runtime."))
print()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


R-squared: 0.3568007295451401
Mean absolute error: 0.6416389395914556
Mean squared error: 0.8269381597435779
Root mean squared error: 0.9093614021628463
=================== Irrelevant answers ==========================
2.897167617455096
Cosine  0.14655071
1.897167617455096

1.453086525489041
Cosine  -0.016367897
0.45308652548904105

2.870634974206055
Cosine  0.2534617
1.8706349742060548

2.780173678593783
Cosine  0.32066128
1.780173678593783

3.68748518799407
Cosine  0.3012029
2.68748518799407

3.2204192687397604
Cosine  0.56606376
4.220419268739761

2.4229418097225444
Cosine  0.506612
3.4229418097225444

3.9501894883596687
Cosine  0.56753397
4.950189488359669

=================== relevant answers ==========================
4.9001978594859
Cosine  0.46199876
5

4.320114097287814
Cosine  0.5833217
5

4.378520515014515
Cosine  0.70521516
5

4.448256202369251
Cosine  0.6507589
5

3.875730990502816
Cosine  0.60621214
4.8757309905028166

4.129722257868863
Cosine  0.6724781
5

4.871960606126

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.6 MB/s eta 0:00:00


In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import re
import tensorflow_hub as hub
import tensorflow as tf
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

stop_words = set(stopwords.words('english'))

# preprocess text function
def preprocess_text(text):
    # convert to lowercase
    text = text.lower()
    # remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # remove stopwords
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(tokens)
    return text

# load the dataset
data = pd.read_csv('dataset.csv')

# preprocess the data
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X = np.array([preprocess_text(q + ' ' + a) for q, a in zip(data['question'], data['student_answer'])])
y = data['score_avg']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# define the LSTM model
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=False),
    tf.keras.layers.Reshape((512,), input_shape=(1, 512)),
    tf.keras.layers.RepeatVector(32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation='relu')),
    tf.keras.layers.Dense(1, activation='linear')
])

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_error'])

# define early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# fit the model to the training data with early stopping
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=16, callbacks=[early_stop])

# evaluate the model on the testing set
y_pred = model.predict(X_test)

# compute evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# print evaluation metrics
print('R-squared:', r2)
print('Mean absolute error:', mae)
print('Mean squared error:', mse)
print('Root mean squared error:', rmse)

# deploy the model to production
def predict_relevance(question, desired_answer, student_answer):
    # preprocess the text
    question = preprocess_text(question)
    desired_answer = preprocess_text(desired_answer)
    student_answer = preprocess_text(student_answer)

    # concatenate question and student_answer texts
    input_text = question + ' ' + student_answer
    
    # predict the relevance score of the input text
    score = model.predict([input_text])[0][0]
    
    print(score)
    # compute cosine similarity between desired_answer and student_answer texts
    similarity = cosine_similarity(embed([desired_answer]).numpy(), embed([student_answer]).numpy())[0][0]
    print("Cosine ", similarity)
    
    # adjust score based on similarity
    if similarity < 0.4:
        score -= 1
    elif similarity >= 0.4 and similarity < 0.8:
        score += 1
    else:
        score += 2
    
    # ensure score is between 0 and 5
    score = min(max(score, 0), 5)
    
    return score

question = "What is the difference between an array declared as static, and one that is not?"
desired_answer = "The arrays declared as static live throughout the life of the program; that is, they are initialized only once, when the function that declares the array it is first called."
student_answer = "a static array will store the new values that were assigned to each of its elements. meaning if you call a function twice it will use the last values that were returned the first time.  if you don't declare it static then the new values will not be stored and will be reset to their original value"
new_answer = "adt adt adt adt adt adt"
newAnswer = "An array declared as static has a fixed size and retains its value between function calls, while a non-static array does not retain its value and its size can change during runtime."
print("=================== Irrelevant answers ==========================")
print(predict_relevance(question, desired_answer, new_answer))
print()
print(predict_relevance(question, desired_answer, "I don't know"))
print()
print(predict_relevance(question, desired_answer, "static one stays while non static one leaves"))
print()
print(predict_relevance(question, desired_answer, "one is memory allocated while other is storage allocated"))
print()
print(predict_relevance(question, desired_answer, "The static array is like a classic car that never goes out of style, while the non-static array is like a trendy new smartphone that quickly becomes outdated."))
print()
print(predict_relevance(question, desired_answer, "static array, non-static array, static array non-static array static array non-static array static array non-static array static array non-static array static array non-static array "))
print()
print(predict_relevance(question, desired_answer, "An array is something which starts with square brackets and all values inside it are seaperated by commas thats why it is static"))
print()
print(predict_relevance(question, desired_answer, "The static array is something that is not non static. Thats why it is called as static array."))
print()


print("=================== relevant answers ==========================")
print(predict_relevance(question, desired_answer, student_answer))
print()
print(predict_relevance(question, desired_answer, newAnswer))
print()
print(predict_relevance(question, desired_answer, "A static array persists throughout the lifetime of a program, while a non-static array is deallocated once it goes out of scope."))
print()
print(predict_relevance(question, desired_answer, "A static array is initialized to zero by default, while a non-static array has an undefined initial value unless explicitly initialized."))
print()
print(predict_relevance(question, desired_answer, "A static array can be accessed from any function within a file, while a non-static array is only accessible within the function in which it is declared."))
print()
print(predict_relevance(question, desired_answer, "A static array can be initialized only once, at compile time, while a non-static array can be initialized multiple times, including at runtime."))
print()
print(predict_relevance(question, desired_answer, "A static array can have a fixed size that is determined at compile time, while a non-static array can have a variable size that is determined at runtime."))
print()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Epoch 1/10
91/91 [==============================] - 53s 353ms/step - loss: 2.4640 - mean_absolute_error: 1.1762 - mean_squared_error: 2.4640 - val_loss: 0.8494 - val_mean_absolute_error: 0.7248 - val_mean_squared_error: 0.8494
Epoch 2/10
91/91 [==============================] - 27s 295ms/step - loss: 0.9428 - mean_absolute_error: 0.7476 - mean_squared_error: 0.9428 - val_loss: 0.7503 - val_mean_absolute_error: 0.6589 - val_mean_squared_error: 0.7503
Epoch 3/10
91/91 [==============================] - 27s 300ms/step - loss: 0.7731 - mean_absolute_error: 0.6652 - mean_squared_error: 0.7731 - val_loss: 0.7300 - val_mean_absolute_error: 0.6601 - val_mean_squared_error: 0.7300
Epoch 4/10
91/91 [==============================] - 28s 308ms/step - loss: 0.6528 - mean_absolute_error: 0.6076 - mean_squared_error: 0.6528 - val_loss: 1.2641 - val_mean_absolute_error: 0.9719 - val_mean_squared_error: 1.2641
Epoch 5/10
91/91 [==============================] - 27s 293ms/step - loss: 0.6238 - mean_abs

In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import re
import tensorflow_hub as hub
import tensorflow as tf
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras import regularizers

stop_words = set(stopwords.words('english'))

# preprocess text function
def preprocess_text(text):
    # convert to lowercase
    text = text.lower()
    # remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # remove stopwords
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(tokens)
    return text

# load the dataset
data = pd.read_csv('dataset.csv')

# preprocess the data
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X = np.array([preprocess_text(q + ' ' + a) for q, a in zip(data['question'], data['student_answer'])])
y = data['score_avg']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# define the LSTM model
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=False),
    tf.keras.layers.Reshape((512,), input_shape=(1, 512)),
    tf.keras.layers.RepeatVector(32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.01))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))),
    tf.keras.layers.Dense(1, activation='linear')
])

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_error'])

# define early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# fit the model to the training data with early stopping
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=16, callbacks=[early_stop])

# evaluate the model on the testing set
y_pred = model.predict(X_test)

# compute evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# print evaluation metrics
print('R-squared:', r2)
print('Mean absolute error:', mae)
print('Mean squared error:', mse)
print('Root mean squared error:', rmse)

# deploy the model to production
def predict_relevance(question, desired_answer, student_answer):
    # preprocess the text
    question = preprocess_text(question)
    desired_answer = preprocess_text(desired_answer)
    student_answer = preprocess_text(student_answer)

    # concatenate question and student_answer texts
    input_text = question + ' ' + student_answer
    
    # predict the relevance score of the input text
    score = model.predict([input_text])[0][0]
    print(score)
    
    # compute cosine similarity between desired_answer and student_answer texts
    similarity = cosine_similarity(embed([desired_answer]).numpy(), embed([student_answer]).numpy())[0][0]
    print("Cosine ", similarity)
    
    # adjust score based on similarity
    if similarity < 0.4:
        score -= 1
    elif similarity >= 0.4 and similarity < 0.8:
        score += 1
    else:
        score += 2
    
    # ensure score is between 0 and 5
    score = min(max(score, 0), 5)
    
    return score

question = "What is the difference between an array declared as static, and one that is not?"
desired_answer = "The arrays declared as static live throughout the life of the program; that is, they are initialized only once, when the function that declares the array it is first called."
student_answer = "a static array will store the new values that were assigned to each of its elements. meaning if you call a function twice it will use the last values that were returned the first time.  if you don't declare it static then the new values will not be stored and will be reset to their original value"
new_answer = "adt adt adt adt adt adt"
newAnswer = "An array declared as static has a fixed size and retains its vealue between function calls, while a non-static array does not retain its value and its size can change during runtime."
print("=================== Irrelevant answers ==========================")
print(predict_relevance(question, desired_answer, new_answer))
print()
print(predict_relevance(question, desired_answer, "I don't know"))
print()
print(predict_relevance(question, desired_answer, "static one stays while non static one leaves"))
print()
print(predict_relevance(question, desired_answer, "one is memory allocated while other is storage allocated"))
print()
print(predict_relevance(question, desired_answer, "The static array is like a classic car that never goes out of style, while the non-static array is like a trendy new smartphone that quickly becomes outdated."))
print()
print(predict_relevance(question, desired_answer, "static array, non-static array, static array non-static array static array non-static array static array non-static array static array non-static array static array non-static array "))
print()
print(predict_relevance(question, desired_answer, "An array is something which starts with square brackets and all values inside it are seaperated by commas thats why it is static"))
print()
print(predict_relevance(question, desired_answer, "The static array is something that is not non static. Thats why it is called as static array."))
print()


print("=================== relevant answers ==========================")
print(predict_relevance(question, desired_answer, student_answer))
print()
print(predict_relevance(question, desired_answer, newAnswer))
print()
print(predict_relevance(question, desired_answer, "A static array persists throughout the lifetime of a program, while a non-static array is deallocated once it goes out of scope."))
print()
print(predict_relevance(question, desired_answer, "A static array is initialized to zero by default, while a non-static array has an undefined initial value unless explicitly initialized."))
print()
print(predict_relevance(question, desired_answer, "A static array can be accessed from any function within a file, while a non-static array is only accessible within the function in which it is declared."))
print()
print(predict_relevance(question, desired_answer, "A static array can be initialized only once, at compile time, while a non-static array can be initialized multiple times, including at runtime."))
print()
print(predict_relevance(question, desired_answer, "A static array can have a fixed size that is determined at compile time, while a non-static array can have a variable size that is determined at runtime."))
print()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/10
91/91 [==============================] - 50s 356ms/step - loss: 71.0695 - mean_absolute_error: 3.1224 - mean_squared_error: 57.9115 - val_loss: 13.1515 - val_mean_absolute_error: 1.3109 - val_mean_squared_error: 2.6152
Epoch 2/10
91/91 [==============================] - 26s 281ms/step - loss: 11.3099 - mean_absolute_error: 0.8978 - mean_squared_error: 1.3152 - val_loss: 10.7195 - val_mean_absolute_error: 0.9213 - val_mean_squared_error: 1.2258
Epoch 3/10
91/91 [==============================] - 27s 292ms/step - loss: 10.2171 - mean_absolute_error: 0.8369 - mean_squared_error: 1.0963 - val_loss: 9.9336 - val_mean_absolute_error: 0.8953 - val_mean_squared_error: 1.1598
Epoch 4/10
91/91 [==============================] - 27s 299ms/step - loss: 9.4970 - mean_absolute_error: 0.7783 - mean_squared_error: 1.0119 - val_loss: 9.3876 - val_mean_absolute_error: 0.7917 - val_mean_squared_error: 1.1813
Epoch 5/10
91/91 [==============================] - 28s 304ms/step - loss: 8.9519 - me